In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 19 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

27 February 2023 ; 12:56:52


# Features, model

In [3]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP']#,'GS'] # GS !!!
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,27,0,9.7,1.6,3.6,0.449,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,1,1
1,Bobby Portis,PF,27,MIL,49,15,26.6,5.9,11.9,0.501,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,1
2,Brook Lopez,C,34,MIL,59,59,30.4,5.6,11.0,0.509,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,0
3,Giannis Antetokounmpo,PF,28,MIL,48,48,32.5,11.1,20.7,0.538,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,0
4,Grayson Allen,SG,27,MIL,56,54,27.6,3.4,7.8,0.440,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,0


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Nikola Jokić,DEN,32.1,10.781481,18.374074,24.8,1,0.866866
1,Joel Embiid,PHI,30.6,7.676596,14.655319,33.1,3,0.562290
2,Luka Dončić,DAL,30.0,8.988462,14.034615,33.1,6,0.520272
3,Giannis Antetokounmpo,MIL,28.4,6.833333,11.104167,31.3,2,0.477612
4,Jayson Tatum,BOS,23.8,5.610526,11.508772,30.4,1,0.392977


In [7]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)